In [1]:
%load_ext autotime

time: 0 ns (started: 2023-12-07 07:41:48 +00:00)


In [2]:
import numpy as np
import subprocess
import os
import pandas as pd
import matplotlib.pyplot as plt
import random
import array
import csv


# import deap packages required
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import winsound

time: 734 ms (started: 2023-12-07 07:41:48 +00:00)


In [3]:
path_binary = "CourseworkExecutables/bit_cam_napier_windows.exe"
path_binary_vis = "CourseworkExecutables/bit_cam_napier_visualisation_windows.exe"

pathList = [ "Instance/MerchistonD._5_cameras.csv", 
            "Instance/NMScotlandFloor5Left_5_cameras.csv", 
            "Instance/NMScotlandFloor5Right_5_cameras.csv", 
            "Instance/Wallstest1_5_cameras.csv", 
            "Instance/Wallstest2_5_cameras.csv"]

time: 0 ns (started: 2023-12-07 07:41:49 +00:00)


In [4]:
nb_cameras = 5
instance_size  = 100
num_cells = instance_size * instance_size

time: 0 ns (started: 2023-12-07 07:41:49 +00:00)


## CW_v2_limitCam.ipynb

In [5]:
def openfile(walls):
    with open(myinst) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for line in csv_reader:
            column=int(line[0])
            row=int(line[1])
            oneD_index = (row * instance_size) + column;
            walls[oneD_index]=1
            walls = np.zeros(instance_size*instance_size)
    return walls


time: 0 ns (started: 2023-12-07 07:41:49 +00:00)


In [6]:
def objective_function(x, instance_size, nb_cameras, instance_file):
    params = ['%.16g' % a for a in x]
    cmd = [path_binary,str(instance_size),str(nb_cameras)]+params+[instance_file]
    s = subprocess.check_output(cmd)
    return float(s)


# Do NOT modify: this checks whether a camera is positioned on top of wall in a solution
def check_walls(solution, inst):
    clashes=0
    for i in range(0, len(solution)):
        if (walls[i] == 1 and solution[i]==1):
            clashes+=1
            
    return(clashes)

time: 0 ns (started: 2023-12-07 07:41:49 +00:00)


In [7]:
def convert_individual_bitstring(individual):
    converted_individual = np.zeros(num_cells)
    for i in range(len(individual)):
        converted_individual[individual[i]] = 1
    return converted_individual


time: 0 ns (started: 2023-12-07 07:41:49 +00:00)


In [8]:
fitness_cache = {}

def eval_function(individual, indpb=None):
    indiv_key = tuple(individual)

    if indiv_key in fitness_cache:
        return fitness_cache[indiv_key],

    solution = convert_individual_bitstring(individual)
    total_cameras = np.sum(solution)

    if instance_file == "":
        cameras_on_walls = 0
    else:
        cameras_on_walls = check_walls(solution, instance_file)

    fitness = 0

    if total_cameras > nb_cameras:
        fitness += num_cells * 10 * (total_cameras - nb_cameras)
    elif total_cameras < nb_cameras:
        fitness += num_cells * 2 * (nb_cameras - total_cameras)

    if cameras_on_walls > 0:
        fitness += num_cells * cameras_on_walls

    if not fitness:
        fitness = objective_function(solution, instance_size, nb_cameras, instance_file)

    fitness_cache[indiv_key] = fitness
    return fitness,

time: 0 ns (started: 2023-12-07 07:41:49 +00:00)


In [9]:
def custom_mutate(individual, num_cells_rangeSet, indpb):
    possible_values = list(num_cells_rangeSet - set(individual))
    for i in range(len(individual)):
        if random.random() < indpb:
            if possible_values:
                unused_value = random.choice(list(possible_values))
                possible_values.remove(unused_value)
                individual[i] = unused_value
    return individual,

time: 0 ns (started: 2023-12-07 07:41:49 +00:00)


In [10]:
# Define the fitness class and create an individual class - set up as a minimization problem
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Create a toolbox
toolbox = base.Toolbox()

# Register 'attr_int' to generate a list of unique integers
toolbox.register("attr_int", random.sample, range(num_cells), nb_cameras)

# Register 'individual' to create an individual
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_int)

# Register 'population' to create a population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register all operators needed with the toolbox
toolbox.register("evaluate", eval_function)
toolbox.register("mate", tools.cxTwoPoint)

# toolbox.register("mutate", tools.mutFlipBit, indpb=0.3)
toolbox.register("mutate", custom_mutate, num_cells_rangeSet=set(range(num_cells)), indpb=0.5)

toolbox.register("select", tools.selTournament, tournsize=2)


time: 0 ns (started: 2023-12-07 07:41:49 +00:00)


In [11]:
ngen = 200
popsize = 50

def main():

    pop = toolbox.population(n=popsize)

    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=1.0, mutpb=0.05, ngen=ngen,
                                   stats=stats, halloffame=hof, verbose=False)

    return pop, log, hof

time: 0 ns (started: 2023-12-07 07:41:49 +00:00)


In [12]:
import time
import gc


def runs(n):
    max_fitness_list = []
    gen_number_list = []
    num_cameras_list = []
    iteration_time_list = []

    for iteration in range(n):
        start_time = time.time()
        pop, log, hof = main()

        best = hof[0].fitness.values[0]
        
        min_fitness_per_gen = log.select("min")  
        
        for i in range(ngen):  # set to ngen
            fit = min_fitness_per_gen[i]
            if fit == best:
                break

        end_time = time.time()
        iteration_time = end_time - start_time

        max_fitness_list.append(best)
        gen_number_list.append(i)
        num_cameras_list.append(len(hof[0]))
        iteration_time_list.append(iteration_time)

        print("Run No.", iteration + 1)
        print("max fitness found is %s at gen %s " % (best, i))
        print("number of cameras is %s" % (len(hof[0])))
        print(hof[0])
        print(f"Iteration {iteration + 1} took {iteration_time:.2f} seconds")
        print()

    return max_fitness_list, gen_number_list, num_cameras_list, iteration_time_list

time: 0 ns (started: 2023-12-07 07:41:49 +00:00)


In [13]:
max_fitness_list_total = list()
gen_number_list_total = list()
num_cameras_list_total = list()
iteration_time_list_total = list()

for path in pathList:
    try:
        fitness_cache = {}
        myinst = path
        instance_file = myinst
        
        
        
        num_cells = instance_size*instance_size
        
        walls = np.zeros(instance_size*instance_size)
        walls = openfile(walls)
        n_iterations  = 15
        max_fitness_list, gen_number_list, num_cameras_list, iteration_time_list = runs(n_iterations)
    
        max_fitness_list_total.append(max_fitness_list)
        gen_number_list_total.append(gen_number_list)
        num_cameras_list_total.append(num_cameras_list)
        iteration_time_list_total.append(iteration_time_list)
    
        print(myinst)
        print(max_fitness_list, gen_number_list, num_cameras_list, iteration_time_list, sep="\n\n", end="\n\n\n\n")
    
    except:
        freq=100
        dur=2000
        winsound.Beep(freq,dur)

Run No. 1
max fitness found is 2267.0 at gen 18 
number of cameras is 5
[5815, 7279, 2029, 1178, 8746]
Iteration 1 took 77.81 seconds

Run No. 2
max fitness found is 2074.0 at gen 8 
number of cameras is 5
[1832, 6813, 2078, 6639, 7779]
Iteration 2 took 74.64 seconds

Run No. 3
max fitness found is 2290.0 at gen 158 
number of cameras is 5
[1979, 6844, 1823, 6413, 6778]
Iteration 3 took 91.26 seconds

Run No. 4
max fitness found is 2474.0 at gen 45 
number of cameras is 5
[8339, 7068, 1532, 1479, 7512]
Iteration 4 took 77.65 seconds

Run No. 5
max fitness found is 2395.0 at gen 21 
number of cameras is 5
[1977, 3138, 7593, 7404, 8351]
Iteration 5 took 83.31 seconds

Run No. 6
max fitness found is 2298.0 at gen 199 
number of cameras is 5
[7775, 7513, 1876, 2737, 7748]
Iteration 6 took 93.73 seconds

Run No. 7
max fitness found is 2534.0 at gen 16 
number of cameras is 5
[8074, 6807, 629, 3187, 7132]
Iteration 7 took 85.37 seconds

Run No. 8
max fitness found is 2278.0 at gen 186 
numbe


KeyboardInterrupt



In [ ]:
num_cameras_list

In [ ]:
print(max_fitness_list_total)

In [ ]:
print(iteration_time_list_total)

In [ ]:
for i in iteration_time_list_total:
    print(np.median(i))

In [ ]:
max_fitness_list_total, gen_number_list_total, num_cameras_list_total, iteration_time_list_total

In [ ]:
import winsound
freq=500
dur=1000
winsound.Beep(freq,dur)

In [ ]:
len(max_fitness_list_total)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch

In [ ]:
labels = ["MerchistonD", "NMScotlandFloor5Left", 
          "NMScotlandFloor5Right", "Wallstest1", 
          "Wallstest2"]

In [ ]:
# Create a custom color palette
custom_palette = sns.color_palette("Set1", n_colors=5)  # Adjust the number of colors as needed

fig, axs = plt.subplots(2, 2)

# Section 1
sns.boxplot(data=max_fitness_list_total, ax=axs[0, 0], palette=custom_palette).set_title('Best Fitness')

# Section 2
sns.boxplot(data=gen_number_list_total, ax=axs[0, 1], palette=custom_palette).set_title('NGS')

# Section 3
sns.boxplot(data=num_cameras_list_total, ax=axs[1, 0], palette=custom_palette).set_title('Number of Cameras')

# Section 4
sns.boxplot(data=iteration_time_list_total, ax=axs[1, 1], palette=custom_palette).set_title('Time Consumed')

# Create custom legend handles with colors
legend_handles = [Patch(color=color, label=label) for color, label in zip(custom_palette, labels)]

# Add legend with custom handles
fig.legend(handles=legend_handles, title='Box Labels', loc='upper right', bbox_to_anchor=(1.35, 0.9))

plt.tight_layout()

fig.suptitle('CW_v4_randomSample.ipynb', y=1.05)

plt.show()

In [ ]:
import math

print("Median")
for i, v in enumerate(labels):
    print("""{}
    Best Fitness: {}
    Number of Generations to find Best Fitness: {}
    Number of cameras: {}
    Time Consumed: {}


    
    """.format(
        v,
        np.median(max_fitness_list_total[i]),
        np.median(gen_number_list_total[i]),
        np.median(num_cameras_list_total[i]),
        math.ceil(np.median(iteration_time_list_total[i]))
        )
    )